In [1]:
import pandas as pd
import numpy as np
import keras
from build_model import build_model

data_x = pd.read_csv('./data/weather/v3/2013/merged/merged_2013_h0.csv', index_col = 'Time')
data_x.columns

Using TensorFlow backend.


Index(['temperature-002', 'cloudiness-002', 'temperature-005',
       'cloudiness-005', 'temperature-015', 'cloudiness-015',
       'temperature-027', 'cloudiness-027', 'temperature-070',
       'cloudiness-070', 'temperature-110', 'cloudiness-110',
       'temperature-120', 'cloudiness-120', 'temperature-130',
       'cloudiness-130', 'temperature-145', 'cloudiness-145',
       'temperature-149', 'cloudiness-149', 'temperature-156',
       'cloudiness-156', 'temperature-168', 'cloudiness-168',
       'temperature-180', 'cloudiness-180', 'temperature-190',
       'cloudiness-190', 'temperature-222', 'cloudiness-222',
       'temperature-240', 'cloudiness-240', 'temperature-255',
       'cloudiness-255', 'temperature-260', 'cloudiness-260',
       'temperature-280', 'cloudiness-280', 'temperature-299',
       'cloudiness-299', 'temperature-434', 'cloudiness-434',
       'temperature-460', 'cloudiness-460', 'temperature-481',
       'cloudiness-481', 'temperature-497', 'cloudiness-497',


In [2]:
x = data_x[['temperature-460', 'cloudiness-460',
            'temperature-481', 'cloudiness-481',
            'temperature-645', 'cloudiness-645',]]

In [3]:
data_y = pd.read_csv('./data/energy/v5/2013/Consumption_2013.csv', index_col = 'Time')
data = x.copy()
data['cons'] = data_y['Auvergne-Rhone-Alpes']
data.head()

,temperature-460,cloudiness-460,temperature-481,cloudiness-481,temperature-645,cloudiness-645,cons
Time,,,,,,,
2013-01-01 00:00:00,8.2,8,9.5,4,7.1,7,ND
2013-01-01 01:00:00,8.4,8,9.5,5,7.1,7,7944
2013-01-01 02:00:00,8.3,8,9.6,7,7.4,8,7882
2013-01-01 03:00:00,8.6,8,9.5,7,7.6,6,7709
2013-01-01 04:00:00,8.5,8,9.5,8,8.0,7,7383


In [4]:
data.drop(data.index[0], inplace = True)

In [5]:
data.head()

,temperature-460,cloudiness-460,temperature-481,cloudiness-481,temperature-645,cloudiness-645,cons
Time,,,,,,,
2013-01-01 01:00:00,8.4,8,9.5,5,7.1,7,7944
2013-01-01 02:00:00,8.3,8,9.6,7,7.4,8,7882
2013-01-01 03:00:00,8.6,8,9.5,7,7.6,6,7709
2013-01-01 04:00:00,8.5,8,9.5,8,8.0,7,7383
2013-01-01 05:00:00,8.5,7,9.4,8,7.9,7,7158


In [6]:
data.isnull().values.any()

False

In [7]:
data.isnull().sum()

temperature-460    0
cloudiness-460     0
temperature-481    0
cloudiness-481     0
temperature-645    0
cloudiness-645     0
cons               0
dtype: int64

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [ ]:
data = pd.DataFrame(data)
data.head()

In [ ]:
data = data.values

'''
from sklearn.preprocessing import scale

data = scale( data, axis=0, with_mean=True, with_std=False, copy=True )
data.head()
'''

x_data = data[:, 0:6]
y_data = data[:, 6]
x_data.shape


import tensorflow as tf

In [ ]:
seq_len = 1000
def data_to_sequence(data, n_prev = seq_len):  

    docX, docY = [], []
    for i in range(len(data)-n_prev):
        docX.append(data[i:i+n_prev, 0:6])
        docY.append(data[i+n_prev, 6])
    alsX = np.array(docX)
    alsY = np.array(docY)

    return alsX, alsY

x_data, y_data = data_to_sequence(data)

print(x_data.shape)
print(y_data.shape)

In [ ]:
model = build_model(features = 6, seq_len = seq_len, out = 1)
model.summary()

In [ ]:
from keras.callbacks import TensorBoard , ModelCheckpoint, ReduceLROnPlateau


tbCallBack = TensorBoard(log_dir ='./logs/', 
                         histogram_freq = 0, 
                         write_graph = True)

filepath = "best_model.hdf5"
best_model = ModelCheckpoint(filepath = filepath, 
                             monitor = 'val_loss', 
                             verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = False, 
                             mode = 'auto', period = 1)


reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.5, patience = 2)

In [ ]:
model.fit(x_data,
          y_data,
          batch_size = 100,
          epochs = 800,
          validation_split = 0.05,
          callbacks = [best_model, tbCallBack, reduce_lr])
